In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
import malaya_speech.train.model.conformer as conformer
import malaya_speech.train.model.transducer as transducer
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
config = malaya_speech.config.transducer_featurizer_config
featurizer = malaya_speech.utils.tf_featurization.STTFeaturizer(**config)

In [5]:
config = malaya_speech.config.conformer_small_encoder_config
small_model = conformer.Model(**config)

In [6]:
small_model.conv_subsampling.time_reduction_factor

4

In [7]:
k = 0.0062169171196134855

i = tf.placeholder(tf.float32, [None, None])
lengths = tf.placeholder(tf.float32, [None])
v = tf.expand_dims(tf.map_fn(featurizer.vectorize, i), -1)
length_v = tf.cast(lengths * k, tf.int32)

In [8]:
v

<tf.Tensor 'ExpandDims:0' shape=(?, ?, 80, 1) dtype=float32>

In [9]:
config = malaya_speech.config.conformer_small_decoder_config
small_transducer = transducer.rnn.Model(small_model, vocabulary_size = 100, **config)

In [10]:
config

{'embed_dim': 320,
 'embed_dropout': 0.1,
 'num_rnns': 1,
 'rnn_units': 320,
 'rnn_type': 'lstm',
 'layer_norm': True,
 'projection_units': 0,
 'joint_dim': 320}

In [11]:
p = tf.placeholder(tf.int32, [None, None])
p_len = tf.placeholder(tf.int32, [None])

In [12]:
small_logits = small_transducer([v, p, p_len], training = True)
small_logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


<tf.Tensor 'transducer/transducer_joint/transducer_joint_vocab/BiasAdd:0' shape=(?, ?, ?, 100) dtype=float32>

In [ ]:
greedy_decoded = small_transducer.greedy_decoder(v)

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [ ]:
y, sr = malaya_speech.load('../speech/example-speaker/husein-zolkepli.wav', sr = 16000)
y1, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav', sr = 16000)
len(y), len(y1)

In [ ]:
padded, lens = malaya_speech.padding.sequence_1d([y, y1], return_len = True)
t = [[1,1,1,1,1,1], [1,1,1,1,1,1]]

In [ ]:
sess.run(small_logits, feed_dict = {i: padded, lengths: lens, p: t}).shape

In [ ]:
%%time

greedy = sess.run(greedy_decoded, feed_dict = {i: padded})
greedy.shape

In [ ]:
greedy[0]